In [43]:
#import Libriaries
import os 
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
import pandasgui as pg 
from sqlalchemy import create_engine
import urllib
import re
import pyodbc 
#import pydash

In [44]:
# Define the connection details
server = 'jacobo-dev.database.windows.net'
port = '1433'
database = 'jacobo-dev-sqlserver-azure-001'
username = 'azure-admin'
password = 'ja-2023-un0ypzjo'
driver = '{ODBC Driver 18 for SQL Server}'

# Define the connection string
conn_str = f"DRIVER={driver};SERVER={server},{port};DATABASE={database};UID={username};PWD={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30"

# Test SQL DB Connection
try:
    conn = pyodbc.connect(conn_str)
    print("Connection successful!")
    conn.close()
except pyodbc.Error as e:
    print("Error connecting to database:", e)

Connection successful!


In [45]:
#functions

def get_root(file_path):
    """Parse an XML file and return its root element."""
    xml_tree = ET.parse(file_path)
    xml_root = xml_tree.getroot()
    return xml_root


def get_child_elements(xml_root):
    # Create a list to store all child elements
    elements = []
    # Get all child elements of a given xml element
    for child in xml_root:
        elements.append(child)
        elements.extend(get_child_elements(child))
    #append xml root to elements list
    #elements.append(xml_root)
    return elements

def get_attribute_cfdi(elements, tag_contains, attribute):
    attribute_values = []
    for element in elements:
        cleaned_tag = re.sub(r"{.*?}", "", element.tag)
        # Check if the element's tag contains the required text and then extract the attribute
        if tag_contains in cleaned_tag:
            attribute_value = element.attrib.get(attribute)
            if attribute_value is not None:
                attribute_values.append(attribute_value)
    values = ', '.join(attribute_values)      
    return values
#print(get_attribute_cfdi(elements, 'TimbreFiscalDigital', 'UUID'))
#print(get_attribute_cfdi(elements, 'Receptor', 'Rfc'))


def pascal_case(text):
    return ''.join(word.capitalize() for word in text.split())


def get_xml_metadata(elements):

    # Define a list to store row data
    data = []

    # Define a dictionary for the attribute field counters
    field_counters = {}

    # Get UUID from 'TimbreFiscalDigital' element
    uuid = get_attribute_cfdi(elements, 'TimbreFiscalDigital', 'UUID')
    
    # Iterate over elements
    for element in elements:
        if element.attrib:
            cleaned_tag = pascal_case(re.sub(r"{.*?}", "", element.tag)).lower()
            file_name = os.path.basename(xml_file_path)
            file_path = xml_file_path

            # If this tag has not been seen before, initialize its counter
            if element.tag not in field_counters:
                field_counters[element.tag] = 1

            # Iterate over attributes of the element
            for key, value in element.attrib.items():
                cleaned_key = pascal_case(re.sub(r"{.*?}", "", key)).lower()
                # Add the data to the list, including the current attribute field number
                data.append({'field_number': field_counters[element.tag], 'file_path': file_path, 'file_name': file_name, 'cleaned_tag': cleaned_tag, 'tag': element.tag, 'key': key, 'cleaned_key': cleaned_key,'value': value, 'UUID': uuid})

            # Increment the counter for this attribute field
            field_counters[element.tag] += 1

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(data)

    # Return the DataFrame
    return df


def get_unique_tags(elements):
    # Define a list to store tag names
    tag_list = []

    for element in elements:
        # If the attribute dictionary is not empty, add the tag to the list
        if element.attrib:
            tag_list.append(element.tag)

    # Convert the list to a set to get unique tags, then convert it back to a list
    tag_list = list(set(tag_list))
    
    return tag_list


def create_dataframes(elements, tag_list, metadata_df):
    # Create an empty dictionary to hold the dataframes
    cfdi_df = {}

    # key fields
    uuid = get_attribute_cfdi(elements, 'TimbreFiscalDigital', 'UUID')
    emisor_rfc = get_attribute_cfdi(elements, 'Emisor', 'Rfc')
    receptor_rfc = get_attribute_cfdi(elements, 'Receptor', 'Rfc')
    comprobante_tipo = get_attribute_cfdi(elements, 'Comprobante', 'TipoDeComprobante')

    for tag in tag_list:
        # Filter the DataFrame
        filtered_df = metadata_df[metadata_df['tag'] == tag]

        # Custom aggregation function to concatenate values into a list
        aggfunc = lambda x: list(x) if len(x) > 1 else np.max(x)

        # Create a pivot table
        pivot_table = pd.pivot_table(filtered_df, values='value', index=['field_number'], columns=['cleaned_key'], aggfunc=aggfunc)

        # Reset index and change the column names
        df = pivot_table.reset_index().rename_axis(None, axis=1)

        # Add the UUID as a new column to the DataFrame
        df['uuid'] = uuid
        df['emisor_rfc'] = emisor_rfc
        df['receptor_rfc'] = receptor_rfc
        df['comprobante_tipo'] = comprobante_tipo

        cfdi_df[tag] = df
        
    return cfdi_df


# Function to create a connection engine
def create_db_engine(driver, server, port, database, username, password):
    params = urllib.parse.quote_plus(
        f'DRIVER={driver};SERVER={server},{port};DATABASE={database};UID={username};PWD={password}'
    )
    engine = create_engine(f'mssql+pyodbc:///?odbc_connect={params}')
    return engine


# Function to write a DataFrame to a SQL table
def write_to_db(df_dict, engine):
    for tag, df in df_dict.items():
        cleaned_tag = re.sub(r"{.*?}", "", tag)
        table_name = 'cfdi_' + cleaned_tag
        df.to_sql(table_name, engine, if_exists='append', index=False)


In [46]:
# Define the XML file path
xml_file_path = r"C:\Users\Roberto\OneDrive\cargoIabono\Proyectos y Desarrollos\P001_V001_CFDI-Reader\01_Inputs\Facturas\0F23FE0D-8324-4BCE-AFAC-68CB67E89714.xml"

# Another XML file path for future use
# xml_file_path = r"C:\Users\Roberto\OneDrive\cargoIabono\Proyectos y Desarrollos\P001_V001_CFDI-Reader\01_Inputs\Nomina\EMS2103108P3_Pago de nómina_20220815_N_AOAR951019842.xml"

# Parse the XML file and get its root element
xml_root = get_root(xml_file_path)

# Get all child elements of the root element
elements = get_child_elements(xml_root)

# Append the root element to the list of child elements
elements.append(xml_root)

# Extract metadata from the elements and convert it into a pandas DataFrame
metadata_df = get_xml_metadata(elements)

# Extract unique tags from the elements
tag_list = get_unique_tags(elements)

# Create a dictionary of pandas DataFrames, each containing data for a unique tag
cfdi_df = create_dataframes(elements, tag_list, metadata_df)

# Create a connection engine for the SQL server
engine = create_db_engine(driver, server, port, database, username, password)

# Write each DataFrame in the dictionary to a separate SQL table
write_to_db(cfdi_df, engine)


In [47]:
metadata_df.head()

,field_number,file_path,file_name,cleaned_tag,tag,key,cleaned_key,value,UUID
0,1,C:\Users\Roberto\OneDrive\cargoIabono\Proyecto...,0F23FE0D-8324-4BCE-AFAC-68CB67E89714.xml,emisor,{http://www.sat.gob.mx/cfd/3}Emisor,Rfc,rfc,FIS780810KQ9,0F23FE0D-8324-4BCE-AFAC-68CB67E89714
1,1,C:\Users\Roberto\OneDrive\cargoIabono\Proyecto...,0F23FE0D-8324-4BCE-AFAC-68CB67E89714.xml,emisor,{http://www.sat.gob.mx/cfd/3}Emisor,Nombre,nombre,"FISACERO, S.A.P.I. DE C.V.",0F23FE0D-8324-4BCE-AFAC-68CB67E89714
2,1,C:\Users\Roberto\OneDrive\cargoIabono\Proyecto...,0F23FE0D-8324-4BCE-AFAC-68CB67E89714.xml,emisor,{http://www.sat.gob.mx/cfd/3}Emisor,RegimenFiscal,regimenfiscal,601,0F23FE0D-8324-4BCE-AFAC-68CB67E89714
3,1,C:\Users\Roberto\OneDrive\cargoIabono\Proyecto...,0F23FE0D-8324-4BCE-AFAC-68CB67E89714.xml,receptor,{http://www.sat.gob.mx/cfd/3}Receptor,Rfc,rfc,FAN540305I15,0F23FE0D-8324-4BCE-AFAC-68CB67E89714
4,1,C:\Users\Roberto\OneDrive\cargoIabono\Proyecto...,0F23FE0D-8324-4BCE-AFAC-68CB67E89714.xml,receptor,{http://www.sat.gob.mx/cfd/3}Receptor,Nombre,nombre,"GENVAMEX, S.A. DE C.V. AV. NOGALAR SUR No. 301",0F23FE0D-8324-4BCE-AFAC-68CB67E89714


In [48]:
print(tag_list)

['{http://www.sat.gob.mx/Pagos}DoctoRelacionado', '{http://www.sat.gob.mx/cfd/3}Comprobante', '{http://www.sat.gob.mx/TimbreFiscalDigital}TimbreFiscalDigital', '{http://www.sat.gob.mx/cfd/3}Emisor', '{http://www.sat.gob.mx/cfd/3}Receptor', '{http://www.sat.gob.mx/Pagos}Pago', '{http://www.sat.gob.mx/cfd/3}Concepto', '{http://www.sat.gob.mx/Pagos}Pagos']


In [49]:
cfdi_df['{http://www.sat.gob.mx/cfd/3}Receptor']

,field_number,nombre,rfc,usocfdi,uuid,emisor_rfc,receptor_rfc,comprobante_tipo
0,1,"GENVAMEX, S.A. DE C.V. AV. NOGALAR SUR No. 301",FAN540305I15,P01,0F23FE0D-8324-4BCE-AFAC-68CB67E89714,FIS780810KQ9,FAN540305I15,P
